# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm",
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = (
    client.chat.completions.create(

        messages=generation_chat_history, model="llama3-70b-8192"

    )
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": mergesort_code})

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array of elements using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has only one element. Then, it merges the two halves using the `merge` function.

The `merge` function takes two sorted arrays as input and merges them into a single sorted array. It does this by comparing elements from each array and adding the smaller one to the result array. If one array is exhausted, it appends the remaining elements from the other array to the result.

This implementation has a time complexity of O(n log n) because the recursion reduces the problem size by half each time, and the merge step takes linear time. The space complexity is O(n) because we need to allocate a new array to store the merged result.

You can test the implementation using a sample array, like this:
```
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or need further clarification!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append({"role": "user", "content": mergesort_code})

Now, let's generate a critique to the Python code.

In [8]:
critique = (
    client.chat.completions.create(

        messages=reflection_chat_history, model="llama3-70b-8192"

    )
    .choices[0]
    .message.content
)

In [9]:
display_markdown(critique, raw=True)

Thank you for sharing your implementation of the Merge Sort algorithm! Overall, your code looks clean, well-structured, and easy to understand. Here are some critiques and recommendations to help improve your code:

**1. Docstring formatting:** Your docstrings are well-written, but they could be formatted better. In Python, it's common to use the triple quotes `"""..."""` for multi-line docstrings. You can also use the `numpy` style docstring format, which is widely adopted in the Python community.

**Recommendation:** Update your docstrings to use the `numpy` style formatting. For example:
```python
def merge_sort(arr):
    """
    Sorts an array of elements using the Merge Sort algorithm.

    Parameters
    ----------
    arr : list
        The array to be sorted

    Returns
    -------
    list
        The sorted array
    """
    ...
```
**2. Type hints:** You've done a great job documenting the function parameters and return types, but you can take it a step further by adding type hints. This will make your code more readable and self-documenting.

**Recommendation:** Add type hints for function parameters and return types. For example:
```python
def merge_sort(arr: list) -> list:
    ...
```
**3. Consistent naming conventions:** Your code uses both camelCase and underscore notation for variable names. It's essential to stick to a consistent naming convention throughout your code.

**Recommendation:** Use underscore notation for variable names, as it's the conventional style in Python.

**4. Redundant comments:** Some of your comments, such as `# The merge_sort function takes an array as input and recursively divides it into two halves until each half has only one element.` are redundant, as they simply restate what the code already does.

**Recommendation:** Remove redundant comments and focus on comments that provide additional context or explain complex logic.

**5. Error handling:** Your implementation assumes that the input array only contains comparable elements. However, if the input array contains non-comparable elements (e.g., strings and integers), your code will raise a `TypeError`.

**Recommendation:** Consider adding input validation to handle such cases. You can raise a `ValueError` or return an error message if the input array contains non-comparable elements.

**6. Testing:** While you've provided an example of how to test the implementation, it's essential to write more comprehensive tests to ensure your code works correctly for various input scenarios.

**Recommendation:** Write additional tests to cover edge cases, such as an empty input array, an array with a single element, or an array with duplicate elements.

Overall, your implementation is well-structured, and with these recommendations, you can make it even more robust and readable. Keep up the good work!

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append({"role": "user", "content": critique})

## Generation Step (II)

In [11]:
essay = (
    client.chat.completions.create(

        messages=generation_chat_history, model="llama3-70b-8192"

    )
    .choices[0]
    .message.content
)

In [12]:
display_markdown(essay, raw=True)

Thank you for the detailed feedback! I appreciate your critiques and recommendations, and I'll address each point to improve my code.

**1. Docstring formatting:** I'll update my docstrings to use the `numpy` style formatting, which is more readable and widely adopted in the Python community.

**Updated code:**
```python
def merge_sort(arr: list) -> list:
    """
    Sorts an array of elements using the Merge Sort algorithm.

    Parameters
    ----------
    arr : list
        The array to be sorted

    Returns
    -------
    list
        The sorted array
    """
    ...
```
**2. Type hints:** I'll add type hints for function parameters and return types to make my code more readable and self-documenting.

**Updated code:**
```python
def merge_sort(arr: list) -> list:
    ...
def merge(left: list, right: list) -> list:
    ...
```
**3. Consistent naming conventions:** I'll stick to the underscore notation for variable names, as it's the conventional style in Python.

**Updated code:**
```python
mid_point = len(arr) // 2
left_half = arr[:mid_point]
right_half = arr[mid_point:]
```
**4. Redundant comments:** I'll remove redundant comments and focus on comments that provide additional context or explain complex logic.

**Updated code:**
```python
# No comment needed here
if len(arr) <= 1:
    return arr
```
**5. Error handling:** I'll add input validation to handle cases where the input array contains non-comparable elements.

**Updated code:**
```python
def merge_sort(arr: list) -> list:
    if not all(isinstance(x, type(arr[0])) for x in arr):
        raise ValueError("Input array must contain comparable elements")
    ...
```
**6. Testing:** I'll write additional tests to cover edge cases, such as an empty input array, an array with a single element, or an array with duplicate elements.

**Updated tests:**
```python
def test_merge_sort():
    assert merge_sort([]) == []
    assert merge_sort([5]) == [5]
    assert merge_sort([5, 2, 8, 3, 1, 6, 4]) == [1, 2, 3, 4, 5, 6, 8]
    assert merge_sort([1, 1, 2, 2, 3, 3]) == [1, 1, 2, 2, 3, 3]
    ...
```
Thank you again for your feedback! I'll make sure to incorporate these improvements to make my code more robust, readable, and maintainable.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from reflection_pattern import ReflectionAgent

In [14]:
agent = ReflectionAgent()

In [15]:
generation_system_prompt = (
    "You are a Python programmer tasked with generating high quality Python code"

)


reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"


user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [16]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=2,
    verbose=1,
)


STEP 1/2

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

### Overview

Merge sort is a popular sorting algorithm that uses a divide-and-conquer approach to sort lists of elements. It works by recursively dividing the list into smaller sublists, sorting each sublist, and then merging the sorted sublists back together.

### Code Implementation

```python
def merge_sort(arr):
    """
    Sorts a list using the merge sort algorithm.

    Args:
        arr (list): The list to be sorted.

    Returns:
        list: The sorted list.
    """
    # Base case: if the list has one or zero elements, it is already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle of the list
    mid = len(arr) // 2

    # Divide the list into two sublists
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the sublists
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the sorted sublists
    return merge(left, right)


def merge(left, rig

## Final result

In [17]:
display_markdown(final_response, raw=True)

**Merge Sort Algorithm Implementation in Python**
=====================================================

### Overview

Merge sort is a popular sorting algorithm that uses a divide-and-conquer approach to sort lists of elements. It works by recursively dividing the list into smaller sublists, sorting each sublist, and then merging the sorted sublists back together.

### Code Implementation

```python
def merge_sort(arr):
    """
    Sorts a list using the merge sort algorithm.

    Args:
        arr (list): The list to be sorted.

    Returns:
        list: The sorted list.
    """
    # Base case: if the list has one or zero elements, it is already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle of the list
    mid = len(arr) // 2

    # Divide the list into two sublists
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the sublists
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the sorted sublists
    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    Args:
        left (list): The first sorted list.
        right (list): The second sorted list.

    Returns:
        list: The merged sorted list.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # If there are remaining elements in either list, append them to the merged list
    merged += left[left_index:]
    merged += right[right_index:]

    return merged


# Example usage:
arr = [64, 34, 25, 12, 22, 11, 90]
print("Original array:", arr)
print("Sorted array:", merge_sort(arr))
```

### Explanation

1.  The `merge_sort` function takes a list as input and recursively divides it into smaller sublists until each sublist has one or zero elements (the base case).
2.  The `merge` function merges two sorted lists into a single sorted list by comparing elements from both lists and adding the smaller element to the merged list.
3.  In the example usage, we create an array `[64, 34, 25, 12, 22, 11, 90]` and sort it using the `merge_sort` function.
4.  The sorted array is then printed to the console.

This implementation of merge sort has a time complexity of O(n log n) and is suitable for large datasets.